In [1]:

import os
import sys

In [21]:

mask_series = frvrs_logs_df.voice_command_command_description.isnull() & frvrs_logs_df.voice_capture_message.isnull()
df = frvrs_logs_df[~mask_series]
# df.sample(min(4, frvrs_logs_df[mask_series].shape[0])).dropna(axis='columns', how='all').T
# df.voice_command_command_description.unique().tolist()

In [18]:

mask_series = (frvrs_logs_df.voice_command_command_description == 'Jocelyn can you take a picture for Hannah')
mask_series |= (frvrs_logs_df.voice_capture_message == 'Jocelyn can you take a picture for Hannah')
df = frvrs_logs_df[mask_series]
df.sample(min(4, frvrs_logs_df[mask_series].shape[0])).dropna(axis='columns', how='all').T

,20287
action_type,VOICE_CAPTURE
elapsed_time,5163490
event_time,2023-05-16 17:41:00
session_uuid,724fdf45-3165-43b8-b9ca-dc07102d2886
file_name,724fdf45-3165-43b8-b9ca-dc07102d2886
logger_version,1.0
time_group,13
voice_capture_message,Jocelyn can you take a picture for Hannah


In [2]:

command_str = f'{sys.executable} -m pip install pip --upgrade'
print(command_str)
!{command_str}

C:\Users\DaveBabbitt\Documents\GitHub\notebooks\nb_env\python.exe -m pip install pip --upgrade
     ---------------------------------------- 2.1/2.1 MB 5.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1


In [4]:

command_str = f'{sys.executable} -m pip install SpeechRecognition --upgrade --quiet'
print(command_str)
!{command_str}

C:\Users\DaveBabbitt\Documents\GitHub\notebooks\nb_env\python.exe -m pip install SpeechRecognition --upgrade --quiet


In [5]:

import platform

platform.system()

'Windows'

In [6]:

platform.python_version()

'3.9.16'

In [7]:

command_str = f'{sys.executable} -m pip install pyaudio --upgrade --quiet'
print(command_str)
!{command_str}

C:\Users\DaveBabbitt\Documents\GitHub\notebooks\nb_env\python.exe -m pip install pyaudio --upgrade --quiet


In [11]:

audio_file_path = os.path.abspath('../data/wav/Harvard_list_01.wav')
command_str = f'"C:\Program Files (x86)\sox-14-4-2\sox.exe" --info {audio_file_path}'
print(command_str)
!{command_str}

"C:\Program Files (x86)\sox-14-4-2\sox.exe" --info C:\Users\DaveBabbitt\Documents\GitHub\notebooks\data\wav\Harvard_list_01.wav

Input File     : 'C:\Users\DaveBabbitt\Documents\GitHub\notebooks\data\wav\Harvard_list_01.wav'
Channels       : 1
Sample Rate    : 48000
Precision      : 25-bit
Duration       : 00:01:07.67 = 3248128 samples ~ 5075.2 CDDA sectors
File Size      : 13.0M
Bit Rate       : 1.54M
Sample Encoding: 32-bit Floating Point PCM



In [10]:

command_str = f'{sys.executable} -m pip install soundfile --upgrade --quiet'
print(command_str)
!{command_str}

C:\Users\DaveBabbitt\Documents\GitHub\notebooks\nb_env\python.exe -m pip install soundfile --upgrade --quiet


In [12]:

import soundfile as sf

data, samplerate = sf.read(audio_file_path)

In [13]:

SoundFile_obj = sf.SoundFile(audio_file_path, 'r+')

In [14]:

print(SoundFile_obj.extra_info)

File : 'C:\Users\DaveBabbitt\Documents\GitHub\notebooks\data\wav\Harvard_list_01.wav' (utf-8 converted from ucs-2)
Length : 13000052
RIFF : 13000044
WAVE
fmt  : 18
  Format        : 0x3 => WAVE_FORMAT_IEEE_FLOAT
  Channels      : 1
  Sample Rate   : 48000
  Block Align   : 4
  Bit Width     : 32
  Bytes/sec     : 192000
data : 12992512
LIST : 78
  INFO
    ICRD : 2018-12-20T15:58:54Z
    ISFT : Adobe Audition CC 2017.1 (Windows)
bext : 602
iXML : 1162
_PMX : 5619
End
**** All non-PCM format files should have a 'fact' chunk.



In [15]:

SoundFile_obj.close()

In [16]:

command_str = f'{sys.executable} -m pip install pydub --upgrade'
print(command_str)
!{command_str}

C:\Users\DaveBabbitt\Documents\GitHub\notebooks\nb_env\python.exe -m pip install pydub --upgrade


In [17]:

import speech_recognition as sr
import re
import soundfile as sf
import tempfile
from pydub import AudioSegment
from pydub.generators import Sine

In [18]:

# Function to generate beep audio segment
def generate_beep(duration, frequency, volume):
    beep = Sine(frequency).to_audio_segment(duration=duration * 1000)
    beep = beep.apply_gain(volume)
    return beep

In [19]:

# Function to replace proper nouns with beeps
def censor_proper_nouns(text, audio_data, sample_rate):
    proper_nouns = re.findall(r'\b[A-Z][a-z]+\b', text)
    audio_segment = AudioSegment(
        audio_data.tobytes(),
        frame_rate=sample_rate,
        sample_width=audio_data.dtype.itemsize,
        channels=len(audio_data.shape)
    )
    for noun in proper_nouns:
        beep_duration = len(noun) * 100  # Adjust the beep duration as needed
        beep_frequency = 1000  # Adjust the beep frequency as needed
        beep_volume = -20  # Adjust the beep volume as needed
        beep = generate_beep(beep_duration, beep_frequency, beep_volume)
        start_time = re.search(r'\b{}\b'.format(noun), text).start() / len(text) * len(audio_segment)
        end_time = start_time + len(beep)
        audio_segment = audio_segment.overlay(beep, position=int(start_time), loop=False)

    modified_audio = audio_segment.get_array_of_samples()
    return modified_audio

In [20]:

# Function to convert speech to text
def transcribe_audio(file_path):
    r = sr.Recognizer()
    audio = sr.AudioFile(file_path)

    with audio as source:
        audio_data = r.record(source)
        text = r.recognize_google(audio_data)

    return text

In [21]:

# Function to save the modified audio file
def save_audio(file_path, audio_data, sample_rate):
    sf.write(file_path, audio_data, sample_rate, format='WAV')

In [22]:

# Function to censor the audio file
def censor_audio(file_path):
    sf_obj = sf.SoundFile(file_path, 'r')

    audio_data = sf_obj.read(dtype='float32')
    sample_rate = sf_obj.samplerate
    sf_obj.close()

    temp_file_path = tempfile.NamedTemporaryFile(suffix=".wav").name
    save_audio(temp_file_path, audio_data, sample_rate)

    text = transcribe_audio(temp_file_path)
    modified_audio = censor_proper_nouns(text, audio_data, sample_rate)


    save_audio(temp_file_path, modified_audio, sample_rate)

    return temp_file_path

In [25]:

modified_audio_file_path = censor_audio(audio_file_path)

In [26]:

print(modified_audio_file_path)

C:\Users\DAVEBA~1\AppData\Local\Temp\tmpc_9ag1j4.wav
